In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

In [ ]:
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

In [ ]:
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
feature_dim = dataset.num_features
num_class = dataset.num_classes
dropout = 0.5

In [ ]:
class GCN(nn.Module):
    def __init__(self, hidden_dim, feature_dim, num_class, dropout=0.5) -> None:
        super().__init__()
        self.gc1 = GCNConv(feature_dim, hidden_dim)
        self.gc2 = GCNConv(hidden_dim, num_class)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = F.relu(self.gc1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gc2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        return x

In [ ]:
model = GCN(hidden_dim=16,
            feature_dim=feature_dim,
            num_class=num_class,
            dropout=0.5
            )
print(model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train(num_epochs):
      model.train()
      for epoch in range(num_epochs):
        out = model(data.x, data.edge_index)  # Perform a single forward pass.
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
        
        optimizer.zero_grad()  # Clear gradients.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
      return loss

train(200)

In [ ]:
model(data.x, data.edge_index)

In [ ]:
def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

In [ ]:
test()

In [ ]:
from models import GCN

In [ ]:
model = GCN(10, 10, 16, 2, 0.5)
print(model)

In [ ]:
import torch
a = torch.Tensor([1,0,1])
b = torch.Tensor([1,1,1])
corr = a==b
